In [4]:
# -*- coding: utf-8 -*-
from os import path
# d = path.dirname(__file__)
d = "/home/mohammad/nlpproject/tweet"
STOPWORD = set([i for i in open((path.join(d, 'stopword.txt'))).read().split('\n')])
# for i in STOPWORD:
#     print i.decode('utf-8')

In [5]:
principlist_text = open('renew_principlist.txt', 'r').read().decode('utf-8')
reformist_text = open('renew_reformist.txt', 'r').read().decode('utf-8')

In [6]:
principlist_words = principlist_text.split()
new_principlist_words = []
reformist_words = reformist_text.split()
new_reformist_words = []

In [7]:
# print principlist_words[1]
# principlist_words[0].endswith('ها'.decode('utf-8'))
# principlist_words.remove(principlist_words[0])
# new_principlist_words.append()

# a = principlist_words[0].split('ها'.decode('utf-8'))
# for i in a: print i

In [8]:
for i in principlist_words:
    if i.endswith('ها'.decode('utf-8')):
        new_principlist_words.append(i.rsplit('ها'.decode('utf-8'),1)[0])
        new_principlist_words.append('ها'.decode('utf-8'))
    elif i.endswith('های'.decode('utf-8')):
        new_principlist_words.append(i.rsplit('های'.decode('utf-8'),1)[0])
        new_principlist_words.append('های'.decode('utf-8'))
#     principlist_words.remove(principlist_words[])
    else:
        new_principlist_words.append(i)
#----------------------------------------
for i in reformist_words:
    if i.endswith('ها'.decode('utf-8')):
        new_reformist_words.append(i.rsplit('ها'.decode('utf-8'),1)[0])
        new_reformist_words.append('ها'.decode('utf-8'))
    elif i.endswith('های'.decode('utf-8')):
        new_reformist_words.append(i.rsplit('های'.decode('utf-8'),1)[0])
        new_reformist_words.append('های'.decode('utf-8'))
#     principlist_words.remove(principlist_words[])
    else:
        new_reformist_words.append(i)

In [11]:
# import enchant
# d = enchant.Dict("en_US")
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
print(isEnglish(u'mechanism'))

for i in new_principlist_words:
    if i in STOPWORD:
        new_principlist_words.remove(i)
    elif isEnglish(i):
        new_principlist_words.remove(i)
#-----------------------------------------
for i in new_reformist_words:
    if i in STOPWORD:
        new_reformist_words.remove(i)
    elif isEnglish(i):
        new_reformist_words.remove(i)
new_principlist_words_train = new_principlist_words[:int(0.9 * float(len(new_principlist_words)))]
new_principlist_words_test = new_principlist_words[int(0.9 * float(len(new_principlist_words))):]

new_reformist_words_train = new_reformist_words[:int(0.9 * float(len(new_reformist_words)))]
new_reformist_words_test = new_reformist_words[int(0.9 * float(len(new_reformist_words))):]

True


In [12]:
principlist_existance = {}
j = 0
for i in new_reformist_words_train:
#     if j%1000 == 10:
#         print j
    if i not in principlist_existance.keys():
         principlist_existance[i] = 1
    else:
        principlist_existance[i] +=1
#     j += 1
#------------------------------

reformist_existance = {}
for i in new_principlist_words_train:
    if i not in reformist_existance.keys():
        reformist_existance[i] = 1
    else:
        reformist_existance[i] +=1

        

In [13]:
total_existance = {}
for i in reformist_existance.keys():
    if i not in principlist_existance.keys():
        total_existance[i] = reformist_existance[i]
    else:
        total_existance[i] = reformist_existance[i] + principlist_existance[i]



In [14]:
import json
reformist = open('reformist.json', 'r')
principlist = open('principlist.json', 'r')

reformist_json = json.load(reformist)
principlist_json = json.load(principlist)

reformist_tweet_count = 0
for i in reformist_json:
    reformist_tweet_count += len(i)

principlist_tweet_count = 0
for i in principlist_json:
    principlist_tweet_count += len(i)

print reformist_tweet_count
print principlist_tweet_count

principlist_tweet_probability = float(principlist_tweet_count)/(principlist_tweet_count + reformist_tweet_count)
reformist_tweet_probability = float(reformist_tweet_count)/(reformist_tweet_count + principlist_tweet_count)
print principlist_tweet_probability
print reformist_tweet_probability

6716
5061
0.429735925957
0.570264074043


In [15]:

total_reformist_distinct = len(reformist_existance.keys())
total_reformist = 0
for i in reformist_existance.keys():
    total_reformist += reformist_existance[i]
    
total_reformist_with_smooth = total_reformist + total_reformist_distinct


total_principlist_distinct = len(principlist_existance.keys())
total_principlist = 0
for i in principlist_existance.keys():
    total_principlist += principlist_existance[i]

total_principlist_with_smooth = total_principlist + total_principlist_distinct


naive = {}
# navie = {word : [
#                  reformist_count, principlist_count,
#                  reformist_naive_bayse_with_smooth, reformist_naive_bayse_without_smooth
#                  principlist_naive_bayse_with_smooth, principlist_naive_bayse_without_smooth  
# ]}

for i in total_existance:
    if total_existance[i] > 10:
        naive[i] = []
        if i in reformist_existance.keys():
            reformist_count = reformist_existance[i]
        else:
            reformist_count = 0
        if i in principlist_existance.keys():
            principlist_count = principlist_existance[i]
        else:
            principlist_count = 0

        naive[i].append(reformist_count)
        naive[i].append(principlist_count)

        # reformist_naive_bayse_with_smooth ~ P(word|reformist)
        reformist_naive_bayse_with_smooth = float(reformist_count + 1) / total_reformist_with_smooth
        naive[i].append(reformist_naive_bayse_with_smooth)
    

        # reformist_naive_bayse_without_smooth ~ P(word|reformist)
        reformist_naive_bayse_without_smooth = float(reformist_count ) / total_reformist
        naive[i].append(reformist_naive_bayse_without_smooth)
        
        
        # principlist_naive_bayse_with_smooth ~ P(word|principlist)
        principlist_naive_bayse_with_smooth = float(principlist_count + 1) / total_reformist_with_smooth
        naive[i].append(principlist_naive_bayse_with_smooth)
    

        # principlist_naive_bayse_without_smooth ~ P(word|principlist)
        principlist_naive_bayse_without_smooth = float(principlist_count ) / total_principlist
        naive[i].append(principlist_naive_bayse_without_smooth)
    
    
    

In [17]:
from pprint import pprint
pprint(naive)

{u'!\u061f': [6,
              6,
              6.416896605461696e-05,
              6.217165594205601e-05,
              6.416896605461696e-05,
              4.472838687072751e-05],
 u'\xab': [326,
           323,
           0.002997607414265678,
           0.0033779933061850436,
           0.002970106428813699,
           0.0024078781598741643],
 u'\xbb': [328,
           324,
           0.003015941404566997,
           0.003398717191499062,
           0.0029792734239643586,
           0.0024153328910192853],
 u'\u060c': [2357,
             3817,
             0.021615774565255257,
             0.024423098842571007,
             0.03499958748521822,
             0.028454708780927814],
 u'\u061b': [392,
             474,
             0.003602629094209209,
             0.00406188152154766,
             0.004354322696563294,
             0.003533542562787473],
 u'\u061f': [246,
             413,
             0.0022642478022129125,
             0.0025490378936242966,
             0.003795

                         0.00010083694665725522,
                         7.45473114512125e-05],
 u'\u0627\u0645\u0646\u06cc\u062a': [90,
                                     100,
                                     0.0008341965587100205,
                                     0.0009325748391308403,
                                     0.000925866510216616,
                                     0.0007454731145121251],
 u'\u0627\u0645\u0646\u06cc\u062a\u06cc': [17,
                                           36,
                                           0.00016500591271187217,
                                           0.00017615302516915871,
                                           0.00033917882057440393,
                                           0.00026837032122436507],
 u'\u0627\u0645\u0648\u0627\u0644': [4,
                                     9,
                                     4.5834975753297824e-05,
                                     4.144777062803735e-05,
                

                               0.00012673042946706128],
 u'\u0628\u0646\u062f\u06cc': [6,
                               7,
                               6.416896605461696e-05,
                               6.217165594205601e-05,
                               7.333596120527652e-05,
                               5.218311801584876e-05],
 u'\u0628\u0646\u0632\u06cc\u0646': [5,
                                     40,
                                     5.5001970903957394e-05,
                                     5.180971328504668e-05,
                                     0.0003758468011770422,
                                     0.00029818924580485],
 u'\u0628\u0646\u06cc\u0627\u062f': [7,
                                     16,
                                     7.333596120527652e-05,
                                     7.253359859906535e-05,
                                     0.0001558389175612126,
                                     0.00011927569832194001],
 u'\u0628\u0646

 u'\u062a\u0648\u0644\u06cc\u062a': [14,
                                     5,
                                     0.00013750492725989348,
                                     0.0001450671971981307,
                                     5.5001970903957394e-05,
                                     3.727365572560625e-05],
 u'\u062a\u0648\u0644\u06cc\u062f': [116,
                                     48,
                                     0.0010725384326271692,
                                     0.001201985348213083,
                                     0.0004491827623823187,
                                     0.00035782709496582005],
 u'\u062a\u0648\u0644\u06cc\u062f\u0627\u062a': [7,
                                                 9,
                                                 7.333596120527652e-05,
                                                 7.253359859906535e-05,
                                                 9.166995150659565e-05,
                                

                                                 5.5001970903957394e-05,
                                                 5.180971328504668e-05,
                                                 6.416896605461696e-05,
                                                 4.472838687072751e-05],
 u'\u062e\u06cc\u0627\u0628\u0627\u0646\u200c': [7,
                                                 4,
                                                 7.333596120527652e-05,
                                                 7.253359859906535e-05,
                                                 4.5834975753297824e-05,
                                                 2.9818924580485003e-05],
 u'\u062e\u06cc\u0631': [12,
                         12,
                         0.00011917093695857435,
                         0.00012434331188411203,
                         0.00011917093695857435,
                         8.945677374145501e-05],
 u'\u062e\u06cc\u0631\u06cc\u0647': [1,
                     

...

                                                 0.00016500591271187217,
                                                 0.00017615302516915871,
                                                 0.00017417290786253173,
                                                 0.00013418516061218253],
 u'\u0642\u0631\u0627\u0631\u06af\u0627\u0647': [4,
                                                 7,
                                                 4.5834975753297824e-05,
                                                 4.144777062803735e-05,
                                                 7.333596120527652e-05,
                                                 5.218311801584876e-05],
 u'\u0642\u0631\u0628\u0627\u0646\u06cc': [2,
                                           9,
                                           2.7500985451978697e-05,
                                           2.0723885314018673e-05,
                                           9.166995150659565e-05,
                     

                               1.833399030131913e-05,
                               7.454731145121251e-06],
 u'\u0645\u0637\u0647\u0631\u06cc': [2,
                                     18,
                                     2.7500985451978697e-05,
                                     2.0723885314018673e-05,
                                     0.00017417290786253173,
                                     0.00013418516061218253],
 u'\u0645\u0638\u0644\u0648\u0645': [7,
                                     9,
                                     7.333596120527652e-05,
                                     7.253359859906535e-05,
                                     9.166995150659565e-05,
                                     6.709258030609127e-05],
 u'\u0645\u0639': [11,
                   3,
                   0.00011000394180791479,
                   0.0001139813692271027,
                   3.666798060263826e-05,
                   2.2364193435363753e-05],
 u'\u0645\u0639\u0627\u0635\

                                                             0.00020723885314018672,
                                                             0.0011733753792844243,
                                                             0.0009467508554303989],
 u'\u0646\u0645\u0627\u06cc\u0646\u062f\u06af\u06cc': [4,
                                                       12,
                                                       4.5834975753297824e-05,
                                                       4.144777062803735e-05,
                                                       0.00011917093695857435,
                                                       8.945677374145501e-05],
 u'\u0646\u0645\u0648\u062f': [10,
                               8,
                               0.00010083694665725522,
                               0.00010361942657009336,
                               8.250295635593608e-05,
                               5.963784916097001e-05],
 u'\u0646\u0645\u0648\u064

                               8.28955412560747e-05,
                               6.416896605461696e-05,
                               4.472838687072751e-05],
 u'\u067e\u06cc\u06af\u06cc\u0631': [2,
                                     15,
                                     2.7500985451978697e-05,
                                     2.0723885314018673e-05,
                                     0.00014667192241055304,
                                     0.00011182096717681877],
 u'\u067e\u06cc\u06af\u06cc\u0631\u06cc': [14,
                                           70,
                                           0.00013750492725989348,
                                           0.0001450671971981307,
                                           0.0006508566556968291,
                                           0.0005218311801584876],
 u'\u0686\u0627\u0631\u0647': [4,
                               9,
                               4.5834975753297824e-05,
                             

                   9.166995150659565e-05,
                   6.709258030609127e-05],
 u'\u06f9\u06f6': [9,
                   19,
                   9.166995150659565e-05,
                   9.325748391308402e-05,
                   0.0001833399030131913,
                   0.00014163989175730376],
 u'\u06f9\u06f7': [5,
                   8,
                   5.5001970903957394e-05,
                   5.180971328504668e-05,
                   8.250295635593608e-05,
                   5.963784916097001e-05],
 u'\u200c': [14,
             13,
             0.00013750492725989348,
             0.0001450671971981307,
             0.00012833793210923392,
             9.691150488657627e-05],
 u'\u2026': [81,
             107,
             0.0007516936023540844,
             0.0008393173552177563,
             0.000990035476271233,
             0.0007976562325279739]}


In [18]:
motahari_tweet = "بیاد دارم که در آستانه پیروزی انقلاب، فردی از پدرم پرسید: اگر اتفاقی برای امام خمینی بیفتد چه کسی برای رهبری مناسب است، ایشان گفت:آقای صدوقی".decode('utf-8')
motahari_words = motahari_tweet.split()

In [19]:
is_reformist = principlist_tweet_probability
is_principlist = reformist_tweet_probability
for i in motahari_words:
    if i in naive.keys():
        
        is_reformist *= naive[i][2]
        is_principlist *= naive[i][4]
print is_reformist
print is_principlist

if is_reformist > is_principlist:
    print "motahari is reformist"
else:
    print "motahari is principlist"

5.8874365785e-47
4.38450868858e-46
motahari is principlist


In [33]:
# new_principlist_words_test
# new_reformist_words_test
reformist_tweets = []
print len(new_reformist_words_test)
for i in range(len(new_reformist_words_test)/30):
    reformist_tweets.append(' '.join(new_reformist_words_test[30 * i:30* i + 30]))
    
principlist_tweets = []
print len(new_principlist_words_test)
for i in range(len(new_principlist_words_test)/30):
    principlist_tweets.append(' '.join(new_principlist_words_test[30 * i:30* i + 30]))



14905
10723


In [53]:
print len(principlist_tweets)
print len(reformist_tweets)
reformist_predict = []
for i in reformist_tweets:
    temp_split = i.split()
    is_reformist = reformist_tweet_probability 
    is_principlist = principlist_tweet_probability
    for j in temp_split:
        
        if j in naive.keys():
        
            is_reformist *= naive[j][2]
            is_principlist *= naive[j][4]
    if is_reformist > is_principlist:
        reformist_predict.append(1)
    else:
        reformist_predict.append(0)
            

            
            
principlist_predict = []
ss = 0
for i in principlist_tweets:
    temp_split = i.split()
    is_reformist = reformist_tweet_probability 
    is_principlist = principlist_tweet_probability
    for j in temp_split:
        if j in naive.keys():
        
            is_reformist *= naive[j][2]
            is_principlist *= naive[j][4]
    if ss%100 == 0:
        print "is_reformist {0} is_priciplist {1}".format(is_reformist, is_principlist)
    if is_reformist > is_principlist:
        principlist_predict.append(1)
    else:
        principlist_predict.append(0)
    ss+=1
        

357
496
is_reformist 1.02282909265e-81 is_priciplist 1.34625427097e-79
is_reformist 4.70489468939e-78 is_priciplist 2.61340285898e-74
is_reformist 1.36731328259e-71 is_priciplist 6.62812367538e-74
is_reformist 2.55769505602e-83 is_priciplist 1.93188646584e-82


In [54]:
print principlist_tweet_probability
# precision_reformist = 
# passisive is principlist
# negative is reformist
tp = 0
for i in principlist_predict:
    if i == 1:
        tp +=1
fp = len(principlist_predict) - tp

print len(reformist_predict)
tn = 0
for i in reformist_predict:
    if i == 1:
        tn += 1
fn = len(reformist_predict) - tn
print tp
print principlist_predict[:20]
print tp,tn,fp,fn

0.429735925957
496
32
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
32 23 325 473


In [55]:
precision = float(tp)/(tp + fp)
recall = float(tp) / (tp + fn)
print float(precision)
print float(recall)

0.0896358543417
0.0633663366337
